In [ ]:
import json

# --- Carregar dicionários iniciais e juntar num único dicionário ---

with open('dici_chlo.json', 'r', encoding='utf-8') as f:
    dict_chlo = json.load(f)

with open('dici_covid_hluz.json', 'r', encoding='utf-8') as f:
    dict_covid = json.load(f)

try:
    with open('dicionario_juntar.json', 'r', encoding='utf-8') as f:
        dict_atual = json.load(f)
        print(f"Dicionário inicial carregado com {len(dict_atual)} termos.")
except FileNotFoundError:
    dict_atual = {}
    print("Dicionário inicial não encontrado. Criando um novo.")



def find_existing_term(dictionary, term):
    if term in dictionary:
        return term
    term_lower = term.lower()
    for existing_term in dictionary.keys():
        if existing_term.lower() == term_lower:
            return existing_term
    return None

def add_entries(main_dict, term, content):
    term = term.lower()
    existing_term = find_existing_term(main_dict, term)
    if existing_term:
        if isinstance(main_dict[existing_term], list):
            if content not in main_dict[existing_term]:
                main_dict[existing_term].append(content)
        else:
            if main_dict[existing_term] != content:
                main_dict[existing_term] = [main_dict[existing_term], content]
    else:
        main_dict[term.capitalize()] = content

# Juntar os dicionários
for term, content in dict_chlo.items():
    add_entries(dict_atual, term, content)

for term, content in dict_covid.items():
    add_entries(dict_atual, term, content)

print(f"Total termos após junção: {len(dict_atual)}")


#processar e normalizar campos dos dicionários
for termo, entradas in dict_atual.items():
    if isinstance(entradas, dict):
        entradas = [entradas]
        dict_atual[termo] = entradas

    tem_categoria = any("Categoria" in entrada for entrada in entradas)

    if tem_categoria:
        desc_pt_vals = []
        defin_vals = []
        for entrada in entradas:
            # Normalizar variantes de 'Descricao_pt' / 'descricao_pt'
            if "Descricao_pt" in entrada:
                val = entrada.pop("Descricao_pt")
            elif "descricao_pt" in entrada:
                val = entrada.pop("descricao_pt")
            else:
                val = None
            if val is not None:
                if isinstance(val, list):
                    desc_pt_vals.extend(val)
                else:
                    desc_pt_vals.append(val)

            # Definição
            if "Definição" in entrada:
                val = entrada.pop("Definição")
                if isinstance(val, list):
                    defin_vals.extend(val)
                else:
                    defin_vals.append(val)

        nova_desc = desc_pt_vals + defin_vals

        entradas = [e for e in entradas if "Categoria" in e]

        if entradas and nova_desc:
            entradas[0]["Descricao_pt"] = nova_desc
            dict_atual[termo] = entradas


    else:
        descricoes = []
        outros_campos = {}
        tem_descricao_pt = False
        valor_descricao_pt = None

        for entrada in entradas:
            # Se existir descricao_pt/Descricao_pt, guarda para processar depois
            for key in ["descricao_pt", "Descricao_pt"]:
                if key in entrada:
                    tem_descricao_pt = True
                    valor_descricao_pt = entrada.pop(key)
            # Normalizar todas as variações de descricao para "Descrição"
            for key in list(entrada.keys()):
                if key.lower() == "descricao" and key != "Descrição":
                    if "Descrição" in entrada:
                        val_existente = entrada["Descrição"]
                        val_novo = entrada.pop(key)
                        valores = []
                        if isinstance(val_existente, list):
                            valores.extend(val_existente)
                        else:
                            valores.append(val_existente)
                        if isinstance(val_novo, list):
                            valores.extend(val_novo)
                        else:
                            valores.append(val_novo)
                        valores = [v for i, v in enumerate(valores) if v != "" and v not in valores[:i]]
                        entrada["Descrição"] = valores if len(valores) > 1 else valores[0]
                    else:
                        entrada["Descrição"] = entrada.pop(key)

            for k, v in entrada.items():
                if k not in ["Descrição", "Definição"]:
                    outros_campos[k] = v

            if "Descrição" in entrada:
                val = entrada["Descrição"]
                if isinstance(val, list):
                    descricoes.extend([v for v in val if v != ""])
                else:
                    if val != "":
                        descricoes.append(val)

            if "Definição" in entrada:
                val = entrada["Definição"]
                if isinstance(val, list):
                    descricoes.extend([v for v in val if v != ""])
                else:
                    if val != "":
                        descricoes.append(val)

        # Se só existe descricao_pt/Descricao_pt, passa para Descrição
        if tem_descricao_pt and not descricoes and not outros_campos:
            nova_entrada = {}
            if isinstance(valor_descricao_pt, list):
                nova_entrada["Descrição"] = valor_descricao_pt
            else:
                nova_entrada["Descrição"] = [valor_descricao_pt]
        else:
            nova_entrada = dict(outros_campos)
            # Junta descricao_pt/Descricao_pt a descricoes, se existirem
            if tem_descricao_pt:
                if isinstance(valor_descricao_pt, list):
                    descricoes.extend([v for v in valor_descricao_pt if v != ""])
                else:
                    if valor_descricao_pt != "":
                        descricoes.append(valor_descricao_pt)
            nova_entrada["Descrição"] = descricoes if descricoes else []

        dict_atual[termo] = [nova_entrada]


# Remover "Traducoes" se existir com "Traduções" ---
for termo, entradas in dict_atual.items():
    if isinstance(entradas, dict):
        entradas = [entradas]
        dict_atual[termo] = entradas
    for entrada in entradas:
        if "Traduções" in entrada and "traducoes" in entrada:
            entrada.pop("traducoes")


# Renomeia qualquer "traducoes" ou "Traducoes" para "Traduções"
for termo, entradas in dict_atual.items():
    if isinstance(entradas, dict):
        entradas = [entradas]
        dict_atual[termo] = entradas
    for entrada in entradas:
        for key in ["traducoes", "Traducoes"]:
            if key in entrada:
                entrada["Traduções"] = entrada.pop(key)
                
# passar sigla para lista em todos os casos
for termo, entradas in dict_atual.items():
    if isinstance(entradas, dict):
        entradas = [entradas]
        dict_atual[termo] = entradas
    for entrada in entradas:
        if "sigla" in entrada:
            entrada["Sigla"] = entrada.pop("sigla")
        if "Sigla" in entrada and not isinstance(entrada["Sigla"], list):
            sigla = entrada["Sigla"]
            if isinstance(sigla, str) and sigla.strip():
                entrada["Sigla"] = [sigla]
            else:
                entrada["Sigla"] = []


for termo, entradas in dict_atual.items():
    if isinstance(entradas, dict):
        entradas = [entradas]
        dict_atual[termo] = entradas
    for entrada in entradas:
        if "Descrição" in entrada and not isinstance(entrada["Descrição"], list):
            entrada["Descrição"] = [entrada["Descrição"]]

with open('dicionario_processado.json', 'w', encoding='utf-8') as f:
    json.dump(dict_atual, f, ensure_ascii=False, indent=4)

print("Processamento completo. Dicionário final guardado em 'dicionario_final.json'.")


Dicionário inicial carregado com 3048 termos.
Total termos após junção: 3365
Processamento completo. Dicionário final guardado em 'dicionario_final.json'.


In [ ]:
#Alterado à mão - Sinónimo Complementar para Remissivas

#Caso Bronquiolite - descricao => Descricao_pt

In [ ]:
import json

with open('dicionario_processado.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

flattened = {}

for termo, entradas in data.items():
    if isinstance(entradas, list):
        merged = {}
        for ent in entradas:

            for k, v in ent.items():
                if k in merged:
                    if isinstance(merged[k], list) and isinstance(v, list):
                        for item in v:
                            if item not in merged[k]:
                                merged[k].append(item)
                    elif isinstance(merged[k], list):
                        if v not in merged[k]:
                            merged[k].append(v)
                    elif isinstance(v, list):
                        merged[k] = [merged[k]] + [x for x in v if x != merged[k]]
                    elif merged[k] != v:
                        merged[k] = [merged[k], v]
                else:
                    merged[k] = v
        flattened[termo] = merged

    elif isinstance(entradas, dict):
        flattened[termo] = entradas

with open('dicionario_final.json', 'w', encoding='utf-8') as f:
    json.dump(flattened, f, ensure_ascii=False, indent=4)

In [ ]:

#Sinónimo: "" e Sinónimos: [] => passou a Sinónimos: []

import json

def normalize_sinonimos(entrada):
    """Normalize synonym fields in a dictionary entry"""
    sinonimos = []
    
    for key in ["Sinónimo", "Sinonimo", "sinónimo", "sinonimo", "Sinónimos", "Sinonimos"]:
        if key in entrada:
            value = entrada.pop(key)  
            
            if isinstance(value, str):
                if value.strip(): 
                    sinonimos.append(value.strip())
            elif isinstance(value, list):
                sinonimos.extend([s.strip() for s in value if isinstance(s, str) and s.strip()])
    
    if sinonimos:
        entrada["Sinónimos"] = sinonimos
    
    return entrada

with open('dicionario_final.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

flattened = {}

for termo, entradas in data.items():
    if isinstance(entradas, list):
        merged = {}
        for ent in entradas:
            ent = normalize_sinonimos(ent)
            
            for k, v in ent.items():
                if k in merged:
                    if isinstance(merged[k], list) and isinstance(v, list):
                        for item in v:
                            if item not in merged[k]:
                                merged[k].append(item)
                    elif isinstance(merged[k], list):
                        if v not in merged[k]:
                            merged[k].append(v)
                    elif isinstance(v, list):
                        merged[k] = [merged[k]] + [x for x in v if x != merged[k]]
                    elif merged[k] != v:
                        merged[k] = [merged[k], v]
                else:
                    merged[k] = v
        flattened[termo] = merged

    elif isinstance(entradas, dict):
        flattened[termo] = normalize_sinonimos(entradas.copy())

with open('dicionario_final.json', 'w', encoding='utf-8') as f:
    json.dump(flattened, f, ensure_ascii=False, indent=4)

print("Dicionário processado e salvo com sucesso!")




Dicionário processado e salvo com sucesso!
